In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"desing","key":"313d7b4a9810658d68a8ebaba93dbcc5"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


In [3]:
!kaggle datasets download -d iabhishekofficial/mobile-price-classification

Dataset URL: https://www.kaggle.com/datasets/iabhishekofficial/mobile-price-classification
License(s): unknown
  0% 0.00/70.6k [00:00<?, ?B/s]
100% 70.6k/70.6k [00:00<00:00, 50.7MB/s]


In [4]:
!mkdir mobile-price-classification
!unzip mobile-price-classification.zip -d mobile-price-classification
!ls mobile-price-classification

Archive:  mobile-price-classification.zip
  inflating: mobile-price-classification/test.csv  
  inflating: mobile-price-classification/train.csv  
test.csv  train.csv


In [5]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

import tensorflow as tf
from keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import plot_model
from keras.utils import to_categorical

In [6]:
df = pd.read_csv("/content/mobile-price-classification/train.csv")

In [7]:
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [8]:
df.shape

(2000, 21)

In [9]:
df.columns

Index(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range'],
      dtype='object')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  2000 non-null   int64  
 1   blue           2000 non-null   int64  
 2   clock_speed    2000 non-null   float64
 3   dual_sim       2000 non-null   int64  
 4   fc             2000 non-null   int64  
 5   four_g         2000 non-null   int64  
 6   int_memory     2000 non-null   int64  
 7   m_dep          2000 non-null   float64
 8   mobile_wt      2000 non-null   int64  
 9   n_cores        2000 non-null   int64  
 10  pc             2000 non-null   int64  
 11  px_height      2000 non-null   int64  
 12  px_width       2000 non-null   int64  
 13  ram            2000 non-null   int64  
 14  sc_h           2000 non-null   int64  
 15  sc_w           2000 non-null   int64  
 16  talk_time      2000 non-null   int64  
 17  three_g        2000 non-null   int64  
 18  touch_sc

In [11]:
df.isnull().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [12]:
df['price_range'].value_counts()

price_range
1    500
2    500
3    500
0    500
Name: count, dtype: int64

In [13]:
X = df.drop("price_range", axis=1)
y = df["price_range"]

In [14]:
scaler = MinMaxScaler()
x_normalized = scaler.fit_transform(X)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(x_normalized, y, test_size=0.2)

In [16]:
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (1600, 20)
Shape of y_train: (1600,)
Shape of X_test: (400, 20)
Shape of y_test: (400,)


In [17]:
# Convert labels to one-hot encoding
y_train_cat = to_categorical(y_train, num_classes=4)
y_test_cat = to_categorical(y_test, num_classes=4)

In [18]:
# Define the model
model = Sequential()
model.add(Dense(8, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(6, activation='relu'))
model.add(Dense(4, activation='softmax'))

In [19]:
# Print model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 168       
                                                                 
 dense_1 (Dense)             (None, 6)                 54        
                                                                 
 dense_2 (Dense)             (None, 4)                 28        
                                                                 
Total params: 250 (1000.00 Byte)
Trainable params: 250 (1000.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [21]:
# Train the model
history = model.fit(X_train,
                    y_train_cat,
                    epochs=50,
                    validation_split=0.25)

Epoch 1/50
38/38 [==============================] - 4s 19ms/step - loss: 1.4145 - accuracy: 0.2383 - val_loss: 1.4015 - val_accuracy: 0.2300
Epoch 2/50
38/38 [==============================] - 0s 11ms/step - loss: 1.3893 - accuracy: 0.2725 - val_loss: 1.3828 - val_accuracy: 0.2600
Epoch 3/50
38/38 [==============================] - 0s 8ms/step - loss: 1.3751 - accuracy: 0.2750 - val_loss: 1.3691 - val_accuracy: 0.3175
Epoch 4/50
38/38 [==============================] - 0s 8ms/step - loss: 1.3628 - accuracy: 0.2842 - val_loss: 1.3532 - val_accuracy: 0.3150
Epoch 5/50
38/38 [==============================] - 0s 4ms/step - loss: 1.3471 - accuracy: 0.3050 - val_loss: 1.3277 - val_accuracy: 0.3300
Epoch 6/50
38/38 [==============================] - 0s 3ms/step - loss: 1.3233 - accuracy: 0.3308 - val_loss: 1.2976 - val_accuracy: 0.3575
Epoch 7/50
38/38 [==============================] - 0s 3ms/step - loss: 1.2897 - accuracy: 0.3592 - val_loss: 1.2558 - val_accuracy: 0.3900
Epoch 8/50
38/38 [

In [22]:
# Evaluasi model
loss, accuracy = model.evaluate(X_test, y_test_cat)
print(f"Loss: {loss}, Accuracy: {accuracy}")

13/13 [==============================] - 0s 3ms/step - loss: 0.2783 - accuracy: 0.9200
Loss: 0.2782798707485199, Accuracy: 0.9200000166893005


In [23]:
y_pred_prob = model.predict(X_test)

13/13 [==============================] - 0s 2ms/step


In [24]:
# Convert predicted probabilities to class labels
y_pred = np.argmax(y_pred_prob, axis=1)

In [25]:
# Evaluate model performance
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.93      0.96       107
           1       0.88      0.91      0.89        93
           2       0.89      0.86      0.87        90
           3       0.93      0.96      0.95       110

    accuracy                           0.92       400
   macro avg       0.92      0.92      0.92       400
weighted avg       0.92      0.92      0.92       400

Accuracy: 0.92


In [26]:
def get_class_label(pred):
    if pred == 0:
        return "Low Price"
    elif pred == 1:
        return "Medium Price"
    elif pred == 2:
        return "High Price"
    elif pred == 3:
        return "Very High Price"
    else:
        return "Unknown"

In [27]:
# Contoh data input baru
input_data = (842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1) # Pastikan jumlah fitur sama dengan data pelatihan

# Konversi data input menjadi numpy array
input_data_as_numpy_array = np.asarray(input_data)

# Reshape data input untuk prediksi (menjadi 2D array)
input_data_reshaped = input_data_as_numpy_array.reshape(1, -1)

# Prediksi menggunakan model
prediction_prob = model.predict(input_data_reshaped)

# Cetak probabilitas prediksi
print(f"Prediction probabilities: {prediction_prob}")

# Ambil indeks kelas dengan probabilitas tertinggi
predicted_class = np.argmax(prediction_prob)

# Klasifikasi berdasarkan kelas yang diprediksi
if predicted_class == 0:
    print("Low Price")
elif predicted_class == 1:
    print("Medium Price")
elif predicted_class == 2:
    print("High Price")
elif predicted_class == 3:
    print("Very High Price")
else:
    print("Unknown")

1/1 [==============================] - 0s 35ms/step
Prediction probabilities: [[0. 0. 0. 1.]]
Very High Price


In [28]:
prediction_prob

array([[0., 0., 0., 1.]], dtype=float32)

In [29]:
# Apply the function to all predictions
y_pred_labels = [get_class_label(pred) for pred in y_pred]

# Example: print the first 10 predictions with their true labels
for i in range(10):
    print(f"Predicted: {y_pred_labels[i]}, Actual: {get_class_label(y_test.iloc[i])}")

Predicted: Very High Price, Actual: Very High Price
Predicted: Low Price, Actual: Low Price
Predicted: High Price, Actual: High Price
Predicted: Low Price, Actual: Low Price
Predicted: Very High Price, Actual: Very High Price
Predicted: High Price, Actual: High Price
Predicted: Very High Price, Actual: Very High Price
Predicted: Very High Price, Actual: Very High Price
Predicted: Low Price, Actual: Low Price
Predicted: Low Price, Actual: Low Price
